<a href="https://colab.research.google.com/github/mahisoni36/Bengali_ai_handwritten_grapheme_Classification/blob/master/Colab_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

# **Load Required Libraries**

In [0]:
import os
import cv2
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import matplotlib.pyplot as plt

# **Download  dataset from Kaggle**

In [0]:
! pip install -q kaggle

In [0]:
from google.colab import files
files.upload()

In [0]:
! mkdir ~/.kaggle

In [0]:
! cp kaggle.json ~/.kaggle/

In [0]:
! kaggle competitions download -c bengaliai-cv19

In [0]:
import os  
os.chdir('/content/')

In [0]:
! ls

In [0]:
!unzip train.csv.zip

In [0]:
!unzip test_image_data_0.parquet.zip
!unzip test_image_data_1.parquet.zip
!unzip test_image_data_2.parquet.zip
!unzip test_image_data_3.parquet.zip

In [0]:
!unzip train_image_data_0.parquet.zip
!unzip train_image_data_1.parquet.zip
!unzip train_image_data_2.parquet.zip
!unzip train_image_data_3.parquet.zip

#**Keras Neural Nework Model**

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense,Conv2D,Flatten,MaxPool2D,Dropout,BatchNormalization, Input

In [0]:
inputs = Input(shape = (137, 236, 1))

model = Conv2D(filters=32, kernel_size=3, padding='SAME', activation='relu', input_shape=(137,236, 1))(inputs)
model = Conv2D(filters=32, kernel_size=3,strides=2, padding='SAME', activation='relu')(model)

model = Conv2D(filters=32, kernel_size=3, padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = MaxPool2D(pool_size=(2, 2))(model)

model = Conv2D(filters=32, kernel_size=5, padding='SAME', activation='relu')(model)
model = Dropout(rate=0.4)(model)

model = Conv2D(filters=64, kernel_size=3, strides=2,padding='SAME', activation='relu')(model)
model = Conv2D(filters=64, kernel_size=3, padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = MaxPool2D(pool_size=(2, 2))(model)

model = Conv2D(filters=64, kernel_size=5, padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Dropout(rate=0.3)(model)

model = Conv2D(filters=128, kernel_size=3,strides=2, padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = MaxPool2D(pool_size=(2, 2))(model)

model = Conv2D(filters=128, kernel_size=5, padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Dropout(rate=0.5)(model)

model = Conv2D(filters=256, kernel_size=3, padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = MaxPool2D(pool_size=(2, 2))(model)

model = Conv2D(filters=256, kernel_size=5, padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Dropout(rate=0.3)(model)

model = Flatten()(model)
model = Dropout(rate=0.3)(model)

dense = Dense(256, activation = "relu")(model)

head_root = Dense(168, activation = 'softmax')(dense)
head_vowel = Dense(11, activation = 'softmax')(dense)
head_consonant = Dense(7, activation = 'softmax')(dense)

model = Model(inputs=inputs, outputs=[head_root, head_vowel, head_consonant])

In [0]:
tf.keras.utils.plot_model(
    model, to_file='model.png', show_shapes=False, show_layer_names=True,
    rankdir='TB', expand_nested=True, dpi=65
)

In [0]:
model.compile('adam','categorical_crossentropy',metrics=['accuracy'])

In [0]:
model.summary()

In [0]:
tag_data= pd.read_csv('train.csv')

In [0]:
tag_data.head()

In [0]:
tag_data = tag_data.drop(['grapheme','image_id'],axis=1)

In [0]:
tag_data_root = tag_data['grapheme_root'].values
tag_data_vowel = tag_data['vowel_diacritic'].values
tag_data_consonant = tag_data['consonant_diacritic'].values

In [0]:
len(tag_data_consonant)

# **Train Model**

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
dictionary = {}
for i in range(4):
    data  = pq.read_table(f'train_image_data_{i}.parquet').to_pandas()
    data = data.drop(['image_id'], axis=1)
    print("\n\n")
    print('=='*150)

    data = data.values.reshape(-1, 137, 236, 1)
    
    Y_train_root = tag_data_root[i*50210:(i+1)*50210]
    Y_train_vowel = tag_data_vowel[i*50210:(i+1)*50210]
    Y_train_consonant = tag_data_consonant[i*50210:(i+1)*50210]
    
    Y_train_root = tf.keras.utils.to_categorical(Y_train_root)
    Y_train_vowel = tf.keras.utils.to_categorical(Y_train_vowel)
    Y_train_consonant = tf.keras.utils.to_categorical(Y_train_consonant)

    x_train, x_test, y_train_root, y_test_root, y_train_vowel, y_test_vowel, y_train_consonant, y_test_consonant = train_test_split(data, Y_train_root, Y_train_vowel, Y_train_consonant, test_size=0.2)
    print(x_test.shape,x_train.shape)
    history = model.fit(x_train, [y_train_root, y_train_vowel, y_train_consonant], batch_size=128,epochs =10 ,validation_data=(x_test,[y_test_root,y_test_vowel,y_test_consonant]))

    if(i==0):
       l = list(history.history.keys())
       for j in l:
         dictionary[j] = list()
    for j in l:
      for k in range(len(history.history[j])):
        dictionary[j].append(history.history[j][k])

    del data
    del Y_train_root
    del Y_train_vowel
    del Y_train_consonant


In [0]:
 from matplotlib import figure 

# **Loss And Accuracy Graphs**

In [0]:
loss_train = dictionary['loss']
loss_val = dictionary['val_loss']
epochs = range(1,41)
plt.figure(figsize=(10,6))
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [0]:
lis = ['Grapheme Root','Vowels','Consonant']
for i in range(1,4):
  loss_train = dictionary['dense_'+str(i)+'_loss']
  loss_val = dictionary['val_dense_'+str(i)+'_loss']
  epochs = range(1,41)
  plt.plot(epochs, loss_train, 'g', label='Training loss of '+lis[i-1])
  plt.plot(epochs, loss_val, 'b', label='validation loss of '+lis[i-1])
  plt.title('Training and Validation loss of '+lis[i-1])
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  plt.show()

In [0]:
lis = ['Grapheme Root','Vowels','Consonant']
for i in range(1,4):
  loss_train = dictionary['dense_'+str(i)+'_accuracy']
  loss_val = dictionary['val_dense_'+str(i)+'_accuracy']
  epochs = range(1,41)
  plt.plot(epochs, loss_train, 'g', label='Training Accuracy of '+lis[i-1])
  plt.plot(epochs, loss_val, 'b', label='validation Accuracy of '+lis[i-1])
  plt.title('Training and Validation Accuracy of '+lis[i-1])
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy--->>')
  plt.legend()
  plt.show()

# **Save Model In Drive**

In [0]:
os.chdir('/content/gdrive/My Drive')

In [0]:
model_json = model.to_json()
with open("model_data.json","w") as json_file:
  json_file.write(model_json)
  model.save("model_data.h5")

## **Load Test Data and Predict Output**

In [0]:
os.chdir('/content/')

In [0]:
preds_dict = {
    'grapheme_root': [],
    'vowel_diacritic': [],
    'consonant_diacritic': []
}

In [0]:
components = ['consonant_diacritic', 'grapheme_root', 'vowel_diacritic']
target=[] 
row_id=[] 

In [0]:
for i in range(4):
    df_test_img = pd.read_parquet('test_image_data_{}.parquet'.format(i)) 
    df_test_img.set_index('image_id', inplace=True)

    X_test = df_test_img.values.reshape(-1,137,236,1)
    
    preds = model.predict(X_test)

    for i, p in enumerate(preds_dict):
        preds_dict[p] = np.argmax(preds[i], axis=1)

    for k,id in enumerate(df_test_img.index.values):  
        for i,comp in enumerate(components):
            id_sample=id+'_'+comp
            row_id.append(id_sample)
            target.append(preds_dict[comp][k])
    del df_test_img
    del X_test

# **OutPut CSV File**

In [0]:
df_sample = pd.DataFrame(
    {
        'row_id': row_id,
        'target':target
    },
    columns = ['row_id','target'] 
)
df_sample.to_csv('submission.csv',index=False)
df_sample.head()